In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pima-indians-diabetes-database/diabetes.csv


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import joblib

# Load the dataset
data = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

# Create BMI_category
def categorize_bmi(bmi):
    if bmi < 18.5:
        return 'Underweight'
    elif 18.5 <= bmi < 25:
        return 'Normal'
    elif 25 <= bmi < 30:
        return 'Overweight'
    else:
        return 'Obese'

data['BMI_category'] = data['BMI'].apply(categorize_bmi)

# Save the updated DataFrame to a CSV file
data.to_csv('diabetes_with_bmi_category.csv', index=False)

# Split data
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Separate features and target
X_train = train_data.drop('Outcome', axis=1)
y_train = train_data['Outcome']
X_val = val_data.drop('Outcome', axis=1)
y_val = val_data['Outcome']

# Preprocessing pipeline
numeric_features = X_train.select_dtypes(include=['number']).columns.tolist()
categorical_features = ['BMI_category']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define models and their parameter grids for GridSearchCV
models = {
    'KNN': (KNeighborsClassifier(), {'model__n_neighbors': list(range(1, 20)), 'model__weights': ['uniform', 'distance']}),
    'Decision Tree': (DecisionTreeClassifier(random_state=42), {'model__max_depth': list(range(1, 20)), 'model__min_samples_split': [2, 5, 10]}),
    'Random Forest': (RandomForestClassifier(random_state=42), {'model__n_estimators': [50, 100, 200, 500], 'model__max_depth': list(range(1, 20)), 'model__min_samples_split': [2, 5, 10]}),
    'Logistic Regression': (LogisticRegression(random_state=42, max_iter=10000), {'model__C': [0.001, 0.01, 0.1, 1, 10, 100], 'model__penalty': ['l1', 'l2']}),
    'SVM': (SVC(random_state=42), {'model__C': [0.1, 1, 10, 100], 'model__kernel': ['linear', 'rbf', 'poly'], 'model__degree': [1, 2, 3]})
}

best_model = None
best_accuracy = 0

# Perform GridSearchCV for each model
for model_name, (model, param_grid) in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    print(f"{model_name} Best Parameters: {grid_search.best_params_}, Accuracy: {accuracy}")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = grid_search.best_estimator_

# Ensemble Model
estimators = [
    ('KNN', KNeighborsClassifier(n_neighbors=5)),
    ('Decision Tree', DecisionTreeClassifier(max_depth=7, random_state=42)),
    ('Random Forest', RandomForestClassifier(n_estimators=200, max_depth=9, random_state=42)),
    ('Logistic Regression', LogisticRegression(C=1, random_state=42, max_iter=10000)),
    ('SVM', SVC(C=10, kernel='linear', probability=True, random_state=42)) # Set probability=True
]

ensemble_model = VotingClassifier(estimators=estimators, voting='soft')
ensemble_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('ensemble', ensemble_model)])
ensemble_pipeline.fit(X_train, y_train)
ensemble_pred = ensemble_pipeline.predict(X_val)
ensemble_accuracy = accuracy_score(y_val, ensemble_pred)
print(f"Ensemble Model Accuracy: {ensemble_accuracy}")

if ensemble_accuracy > best_accuracy:
    best_accuracy = ensemble_accuracy
    best_model = ensemble_pipeline

print(f"Best Model: {best_model.steps[-1][1] if not isinstance(best_model, Pipeline) else 'Ensemble'}, Best Accuracy: {best_accuracy}")

# Save preprocessor and model
joblib.dump(best_model.named_steps['preprocessor'], 'preprocessor.joblib')
if isinstance(best_model, Pipeline):
    if 'ensemble' in best_model.named_steps:
        joblib.dump(best_model.named_steps['ensemble'], 'best_model.joblib')
    else:
        joblib.dump(best_model.named_steps['model'], 'best_model.joblib')
else:
    joblib.dump(best_model, 'best_model.joblib')

# Inference Pipeline
def predict_diabetes(sample_data):
    preprocessor = joblib.load('preprocessor.joblib')
    if isinstance(best_model, Pipeline):
        if 'ensemble' in best_model.named_steps:
            model = joblib.load('best_model.joblib')
        else:
            model = joblib.load('best_model.joblib')
    else:
        model = joblib.load('best_model.joblib')

    sample_df = pd.DataFrame([sample_data])
    sample_df['BMI_category'] = sample_df['BMI'].apply(categorize_bmi)

    processed_sample = preprocessor.transform(sample_df)
    if isinstance(best_model, Pipeline):
        prediction = model.predict(processed_sample)[0]
    else:
        prediction = model.predict(processed_sample)[0]
    return prediction

# Test with 5 samples from validation set
for i in range(5):
    sample = X_val.iloc[i].to_dict()
    prediction = predict_diabetes(sample)
    print(f"Sample {i+1} Prediction: {prediction}, Actual: {y_val.iloc[i]}")


KNN Best Parameters: {'model__n_neighbors': 7, 'model__weights': 'uniform'}, Accuracy: 0.7207792207792207
Decision Tree Best Parameters: {'model__max_depth': 3, 'model__min_samples_split': 2}, Accuracy: 0.7597402597402597
Random Forest Best Parameters: {'model__max_depth': 6, 'model__min_samples_split': 2, 'model__n_estimators': 200}, Accuracy: 0.7597402597402597


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
 

Logistic Regression Best Parameters: {'model__C': 1, 'model__penalty': 'l2'}, Accuracy: 0.7467532467532467
SVM Best Parameters: {'model__C': 10, 'model__degree': 1, 'model__kernel': 'linear'}, Accuracy: 0.7662337662337663
Ensemble Model Accuracy: 0.7662337662337663
Best Model: Ensemble, Best Accuracy: 0.7662337662337663
Sample 1 Prediction: 0, Actual: 0
Sample 2 Prediction: 0, Actual: 0
Sample 3 Prediction: 0, Actual: 0
Sample 4 Prediction: 0, Actual: 0
Sample 5 Prediction: 0, Actual: 0
